In [1]:
!pip install simplefix


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive")


Mounted at /content/drive


In [ ]:
import random
import time
from datetime import datetime, timedelta, timezone
from simplefix import *
import simplefix
import socket
import select
from IPython.utils import io



class FIXClient:
    def __init__(self, host, port, sender_comp_id, target_comp_id):
        self.host = host
        self.port = port
        self.sender_comp_id = sender_comp_id
        self.target_comp_id = target_comp_id
        self.sequence_number = 1
        self.ClOrdID = 1
        self.trade_list = []
        self.new_order_run = False

    # creating a socket connection to the FIX server.
    def connect(self):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((self.host, self.port))

    def disconnect(self):
        self.socket.close()

####################### Trivial functions ####################
#############################################################
    def get_utc_timestamp(self):
      # Get the current time in UTC
      current_time_utc = datetime.utcnow().replace(tzinfo=timezone.utc)

      # Format the timestamp as "YYYYMMDD-HH:MM:SS"
      timestamp_str = current_time_utc.strftime("%Y%m%d-%H:%M:%S")

      return timestamp_str

    def calculate_checksum(self,message):
        # Calculate the modulo 256 checksum of the entire message
        checksum_value = sum(map(ord, message)) % 256

        # Format the checksum as a three-character string
        checksum_str = f"{checksum_value:03}"


        return checksum_str

    def receive_messages(self):
        ready, _, _ = select.select([self.socket], [], [], 5)  # Wait for 5 seconds for a response
        if ready:
            data = self.socket.recv(2048)  # Adjust the buffer size based on your needs
            messages = data.decode('utf-8').split('\x01')  # Assuming SOH (Start of Header) is the delimiter
            return messages
        return None

##################### MAIN FUNCTIONS ##################################
######################################################################

# Send a Logon message to the FIX server after the connection has been established.(8,9,35,49,56,34,52,98,108,10)
    def logon(self):
        msg = FixMessage()
        msg.append_pair(8, "FIX.4.2") #### REQUIREMENT 1: FIX Version: FIX.4.2 ####
        msg.append_pair(35, "A") # A : Logon message
        msg.append_pair(49, self.sender_comp_id)
        msg.append_pair(56, self.target_comp_id)
        msg.append_pair(34, self.sequence_number)
        msg.append_pair(52, self.get_utc_timestamp())
        msg.append_pair(98, 0)
        msg.append_pair(108, 30)       # HeartBtInt
        msg.append_pair(10, self.calculate_checksum(f"8=FIX.4.2|{msg}|"))
        msg.append_pair(141, "Y")  # ResetSeqNumFlag: True --> #### REQUIREMENT 6: Sequence number will be reset when logon for simplicity ####
        print(f"LOGON: {msg}")

        self.socket.sendall(msg.encode())

        # Check if receive any messages from server
        response = self.receive_messages()
        res = []
        if response:
          for msg in response:
            res.append(msg)
          print(f"SERVER LOGON: {res}")
        else:
            print("No response received from the server.")


# Handle messages from server
    def process_message(self, response):
        if response[2] == "35=3": # Reject message --> No impact on stats
            reason = response[11].split("=")[1]
            print(f"Received Reject Message. Reason: {reason}")

        elif response[2] == '35=9': # Order Cancel Reject message --> No impact on stats
            reason = response[11].split("=")[1]
            print(f"Received Order Cancel Reject Message. Reason: {reason}")


        elif response[2] == '35=8': # Execution Report message --> Impact on stats

            print("Received Execution Report Message:", response[2])


# New Order (8, 9,35,49,56,34,52,11,21,55,54,60,40,44,38,10)
    def new_order(self, symbol, side, quantity,price, order_type,ClOrdID):
        self.sequence_number += 1
        msg = FixMessage()
        msg.append_pair(8, "FIX.4.2")
        msg.append_pair(35, "D")
        msg.append_pair(49, self.sender_comp_id)
        msg.append_pair(56, self.target_comp_id)
        msg.append_pair(34, self.sequence_number)
        msg.append_pair(52, self.get_utc_timestamp())
        msg.append_pair(11, ClOrdID) # ClOrdID (Client Order ID)
        msg.append_pair(21, 1)
        msg.append_pair(55, symbol)
        msg.append_pair(54, side)
        msg.append_pair(60, self.get_utc_timestamp())
        msg.append_pair(40, order_type)
        msg.append_pair(44, price)
        msg.append_pair(38, quantity)
        msg.append_pair(10, self.calculate_checksum(f"8=FIX.4.2|{msg}|"))
        print(f"CLIENT NEW ORDER: {msg}")

        self.ClOrdID +=1
        self.socket.sendall(msg.encode())
        self.new_order_run = True

        # Check if receive any messages from server
        response = self.receive_messages()
        res = []
        if response:
          result_str = '|'.join(response)
          print(f"SERVER NEW ORDER: {result_str}")
          # for msg in response:
          #   print(f"SERVER: NEW ORDER: {msg}")
          # return response
        else:
            print("No response received from the server.")

# Order Cancel Request(8，9，35，49，56，34， 52，41， 11，55，54，38, 60，10)
    def order_cancel_request(self, symbol, side, quantity, OrigClOrdID, ClOrdID):
        self.sequence_number += 1

        msg = FixMessage()
        msg.append_pair(8, "FIX.4.2")
        msg.append_pair(35, "F")
        msg.append_pair(49, self.sender_comp_id)
        msg.append_pair(56, self.target_comp_id)
        msg.append_pair(34, self.sequence_number)
        msg.append_pair(52, self.get_utc_timestamp())
        msg.append_pair(41, OrigClOrdID)
        msg.append_pair(37, ClOrdID)
        msg.append_pair(11, ClOrdID)
        msg.append_pair(55, symbol)
        msg.append_pair(54, side)
        msg.append_pair(38, quantity)
        msg.append_pair(60, self.get_utc_timestamp())
        msg.append_pair(10, self.calculate_checksum(f"8=FIX.4.2|{msg}|"))
        print(f"CLIENT ORDER CANCEL REQUEST: {msg}")

        self.socket.sendall(msg.encode())
        self.ClOrdID +=1

        # Check if receive any messages from server
        response = self.receive_messages()
        if response:
          result_str = '|'.join(response)
          print(f"SERVER ORDER CANCEL REQUEST: {result_str}")
          #print(f"SERVER ORDER CANCEL REQUEST: {response}")
          # for msg in response:
          #   print(f"SERVER: ORDER CANCEL REQUEST: {msg}")
          # return response
        else:
            print("No response received from the server.")


# Implemenat and run all the requirements

    def run(self):
       with io.capture_output() as cap:
          try:
            #self.sequence_number = 1
            self.connect()
            self.logon()

            start_time = datetime.strptime(self.get_utc_timestamp(), "%Y%m%d-%H:%M:%S")
            end_time = start_time + timedelta(minutes=5)
            price = 0
            resp_dict={}
            limit_order_price = {
        ('MSFT', '1'): 10,
        ('MSFT', '2'): 20,
        ('MSFT', '5'): 10,
        ('AAPL', '1'): 30,
        ('AAPL', '2'): 40,
        ('AAPL', '5'): 30,
        ('BAC', '1'): 50,
        ('BAC', '2'): 60,
        ('BAC', '5'): 50,
    }

            #Send 1000 random orders within 5 minutes
            while start_time <= end_time and self.ClOrdID < 1002:
              symbols = ['MSFT', 'AAPL', 'BAC']
              sides = ["1", "2", "5" ]  # 1=BUY, 2=SELL; 5 = "SELL SHORT"

              # Randomly select symbol, side, quantity, and price (limit order or market order)
              symbol = random.choice(symbols)
              side = random.choice(sides)
              quantity = random.randint(1, 100)
              # Limit Price OR Market Price
              order_type = random.choice(['1', '2'])
              if order_type == "1": price = None # Market Order
              else:
                for i in limit_order_price:
                  if i == (symbol, side):
                    price = limit_order_price[i]
              # Make New Order
              resp = self.new_order(symbol, side, quantity, price, order_type, ClOrdID = self.ClOrdID)
              if self.new_order_run == True:
                # Randomly cancle order
                cancle_order = random.choice(['0', '1']) # "1" = cacnel coder;
                if cancle_order == "1":
                  respc = self.order_cancel_request(symbol, side,quantity, OrigClOrdID = self.ClOrdID-1, ClOrdID = self.ClOrdID)

          except Exception as e:
              print(f"Error: {e}")
          finally:
            self.disconnect()
       with open('out.txt', 'w') as file:
        file.write(cap.stdout)



if __name__ == "__main__":
    fix_client = FIXClient(
        host='xxxxx.com',
        port=5100,
        sender_comp_id='xxxxxx',
        target_comp_id='xxx'
    )

    fix_client.run()
